In [ ]:
import cv2
import numpy as np
import os
import re

In [ ]:
def make_image_path_pairs(dir_real, dir_generated):
    image_pairs = []

    fake_regex = r'(.*)_fake_B_2.png'
    real_regex = r'(.*)_real_B.png'
    fake_paths = {}
    real_paths = {}
    
    for root, _, fnames in sorted(os.walk(dir_real)):
        for fname in fnames:
            path = os.path.join(root, fname)
            match = re.search(real_regex, fname)
            if match:
                key = match[1]
                if key in fake_paths:
                    fake_path = fake_paths[key]
                    image_pairs.append((fake_path, path))
                    del fake_paths[key]
                    continue
                real_paths[key] = path

    for root, _, fnames in sorted(os.walk(dir_generated)):
        for fname in fnames:
            path = os.path.join(root, fname)
            match = re.search(fake_regex, fname)
            if match:
                key = match[1]
                if key in real_paths:
                    real_path = real_paths[key]
                    image_pairs.append((path, real_path))
                    del real_paths[key]
                    continue
                fake_paths[key] = path
    return image_pairs

def MAE(imageA, imageB):
    abs_error = np.sum(abs(imageA.astype("float") - imageB.astype("float")))
    mean_abs_error = abs_error / float(imageA.shape[0] * imageB.shape[1] * imageB.shape[2])
    return mean_abs_error

def RMSE(imageA, imageB):
    sq_error = np.sum((imageA.astype("float") - imageB.astype("float"))**2)
    mse = sq_error / float(imageA.shape[0] * imageB.shape[1] * imageB.shape[2])
    return np.sqrt(mse)

In [ ]:
real_path = '/content/google_drive/MyDrive/images_datasets/real/'
generated_path = '/content/google_drive/MyDrive/images_datasets/generated/'

image_path_pairs = make_image_path_pairs(real_path, generated_path)
image_pairs = [(cv2.imread(fake), cv2.imread(real)) for fake, real in image_path_pairs]

In [ ]:
l1_distances = []
l2_distances = []
for fake, real in image_pairs:
    l1_distances.append(MAE(fake, real))
    l2_distances.append(RMSE(fake, real))

avg_l1_dist = sum(l1_distances)/len(l1_distances)
avg_l2_dist = sum(l2_distances)/len(l2_distances)
print("Average mean average error (l1 loss) between the {} image pairs: {}".format(len(l1_distances), avg_l1_dist))
print("Average root mean squared error (l2 loss) between the {} image pairs: {}".format(len(l2_distances), avg_l2_dist))